In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import sklearn

2023-03-02 22:24:07.125027: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, MaxPooling1D, Conv1D, Dropout
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input, TimeDistributed, LSTM, Bidirectional

In [3]:
x_train = pd.read_pickle('/Users/nitanshjain/Documents/Projects/Depression_Analysis/AVEC_DAIC_WOZ_2017/data/x_train.pk')
y_train = pd.read_pickle('/Users/nitanshjain/Documents/Projects/Depression_Analysis/AVEC_DAIC_WOZ_2017/data/y_train.pk')

print(x_train.shape)
print(y_train.shape)

(62, 512)
(62,)


In [4]:
y_train

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [5]:
x_train = x_train[...,None]

In [6]:
model = Sequential()

"""
    Default values of Batch Normalization are:
        Momentum defaults to 0.99
        The hyperparameter ε defaults to 0.001
        The hyperparameter β defaults to an all-zeros vector
        The hyperparameter γ defaults to an all-ones vector
"""

model.add(Conv1D(16, (3), padding='same', activation='relu', input_shape=(x_train.shape[1:])))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Conv1D(32, (3), padding='same', activation='relu', input_shape=(x_train.shape[1:])))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Conv1D(64, (3), padding='same', activation='relu', input_shape=(x_train.shape[1:])))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Conv1D(128, (3), padding='same', activation='relu', input_shape=(x_train.shape[1:])))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(MaxPooling1D(pool_size=(2), padding='same'))


model.add(LSTM(32, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))

model.add(LSTM(16, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))


model.add(Flatten())
model.add(Dense(1, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), 
                    bias_regularizer=regularizers.L2(1e-4), 
                    activity_regularizer=regularizers.L2(1e-5), 
                    activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

2023-03-02 22:24:14.477488: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 512, 16)           64        
                                                                 
 batch_normalization (BatchN  (None, 512, 16)          64        
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 512, 16)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 256, 16)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 256, 32)           1568      
                                                                 
 batch_normalization_1 (Batc  (None, 256, 32)          1

In [7]:
batch_size = 4

checkpoint_filepath = '/Users/nitanshjain/Documents/Projects/Depression_Analysis/AVEC_DAIC_WOZ_2017/checkpoints/weights-improvement-BatchNormalization-Dropout-{epoch:02d}-{val_accuracy:.2f}.hdf5'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=175,
            callbacks=[model_checkpoint_callback],
            validation_split=0.4,
            shuffle=True)

Epoch 1/175
 9/10 [==========================>...] - ETA: 0s - loss: 0.9505 - accuracy: 0.4444
Epoch 1: val_accuracy improved from -inf to 0.00000, saving model to /Users/nitanshjain/Documents/Projects/Depression_Analysis/AVEC_DAIC_WOZ_2017/checkpoints/weights-improvement-BatchNormalization-Dropout-01-0.00.hdf5
10/10 [==============================] - 13s 247ms/step - loss: 0.9343 - accuracy: 0.4595 - val_loss: 1.0483 - val_accuracy: 0.0000e+00
Epoch 2/175
 9/10 [==========================>...] - ETA: 0s - loss: 1.0226 - accuracy: 0.5278
Epoch 2: val_accuracy did not improve from 0.00000
10/10 [==============================] - 0s 37ms/step - loss: 1.1076 - accuracy: 0.5135 - val_loss: 1.1326 - val_accuracy: 0.0000e+00
Epoch 3/175
10/10 [==============================] - ETA: 0s - loss: 1.0713 - accuracy: 0.4324
Epoch 3: val_accuracy did not improve from 0.00000
10/10 [==============================] - 0s 43ms/step - loss: 1.0713 - accuracy: 0.4324 - val_loss: 1.1591 - val_accuracy: 0.

# Models Accuracy

```
model = Sequential()

model.add(Conv1D(32, (3), padding='same', activation='relu', input_shape=(x_train_input_shape[1], x_train_input_shape[2])))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(LSTM(32, return_sequences=True))
model.add(LSTM(16, return_sequences=True))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
```

Epoch 15/20
6/6 [==============================] - 315s 52s/step - loss: 0.5692 - accuracy: 0.7178 - val_loss: 0.6434 - val_accuracy: 0.7857